In [2]:
from eval_metrics.glossary import *

import json
def load_jsonl(path):
    data=[]
    with open(path, 'r', encoding='utf-8') as reader:
        for line in reader:
            data.append(json.loads(line))
    return data 

ROOT_PATH = "/data/aofei"
dataset = "Slake"
lora_pred_path = "/data/aofei/hallucination/mitigation/Slake/llava_med/lora/epoch6/inference/pred.jsonl"
bv_pred_path = "/data/aofei/hallucination/mitigation/Slake/llava_med/visual_enhance/lora_0.1_bbox_0.01/epoch6/inference/pred.jsonl"

gt_path = f"{ROOT_PATH}/hallucination/{dataset}/data/test.json"

gt = json.load(open(gt_path, 'r'))
# candidate = json.load(open(args.candidate, 'r'))
lora_pred = load_jsonl(lora_pred_path)
bv_pred = load_jsonl(bv_pred_path)

gt_ids = [item['id'] for item in gt]
pred_ids = [item['question_id'] for item in lora_pred]
bv_pred_ids = [item['question_id'] for item in bv_pred]
num_gt_ids, num_pred_ids = len(gt_ids), len(pred_ids)
print(f'num_gt_ids: {num_gt_ids} || num_pred_ids: {num_pred_ids}')
# import pdb; pdb.set_trace()
assert gt_ids == pred_ids, "please make sure pred and gt are exactly matched"
assert gt_ids == bv_pred_ids, "please make sure pred and gt are exactly matched"

num_gt_ids: 1061 || num_pred_ids: 1061


In [3]:
# the original lora
num_close = 0
correct_ids, wrong_ids = [], []
for gt_item, pred_item in zip(gt, lora_pred):
    try:
        gt_results = gt_item['conversations']
    except:
        gt_results = gt_item['conversatons']
    gt_value = gt_results[1]['value'].lower()
    pred_value = pred_item['text'].lower()

    gt_value = normalize_word(gt_value)
    pred_value = normalize_word(pred_value)

    if gt_item['answer_type'] == 'CLOSED':
        num_close += 1
        _id = pred_item['question_id']
        if 'yes' in pred_value or 'no' in pred_value:
            if gt_value in pred_value:
                correct_ids.append(_id)
            else:
                wrong_ids.append(_id)
        # else:
        #     wrong_ids.append(_id)

In [7]:
len(wrong_ids), len(correct_ids)

(48, 307)

In [8]:
# our method, without MOE
num_close = 0
bv_correct_ids, bv_wrong_ids, bv_close_fail_ids = [], [], []
for gt_item, pred_item in zip(gt, bv_pred):
    try:
        gt_results = gt_item['conversations']
    except:
        gt_results = gt_item['conversatons']
    gt_value = gt_results[1]['value'].lower()
    pred_value = pred_item['text'].lower()

    gt_value = normalize_word(gt_value)
    pred_value = normalize_word(pred_value)

    if gt_item['answer_type'] == 'CLOSED':
        num_close += 1
        _id = pred_item['question_id']
        if 'yes' in pred_value or 'no' in pred_value:
            if gt_value in pred_value:
                bv_correct_ids.append(_id)
            else:
                bv_wrong_ids.append(_id)
        else:
            bv_close_fail_ids.append(_id)
        # else:
        #     nobbox_wrong_ids.append(_id)

In [9]:
len(bv_wrong_ids), len(bv_correct_ids), len(bv_close_fail_ids)

(51, 280, 24)

In [25]:
for i in bv_close_fail_ids:
    print(gt_id_dict[i]["organ"], gt_id_dict[i]["conversations"])

Lung [{'from': 'human', 'value': '<image>\nDoes the picture contain liver?'}, {'from': 'gpt', 'value': 'No'}]
Lung [{'from': 'human', 'value': '<image>\nDoes the picture contain lung?'}, {'from': 'gpt', 'value': 'Yes'}]
Lung [{'from': 'human', 'value': '<image>\nDoes the picture contain heart?'}, {'from': 'gpt', 'value': 'No'}]
Lung [{'from': 'human', 'value': '<image>\nIs the lung healthy?'}, {'from': 'gpt', 'value': 'No'}]
Abdomen [{'from': 'human', 'value': '<image>\nDoes the picture contain kidney?'}, {'from': 'gpt', 'value': 'Yes'}]
Abdomen [{'from': 'human', 'value': '<image>\nDoes the picture contain spleen?'}, {'from': 'gpt', 'value': 'No'}]
Abdomen [{'from': 'human', 'value': '<image>\nDoes the picture contain lung?'}, {'from': 'gpt', 'value': 'No'}]
Abdomen [{'from': 'human', 'value': '<image>\nDoes the picture contain heart?'}, {'from': 'gpt', 'value': 'No'}]
Abdomen [{'from': 'human', 'value': '<image>\nDoes the picture contain liver?'}, {'from': 'gpt', 'value': 'Yes'}]
Abd

In [12]:
common_wrong_ids, bv_wrongs, lora_wrongs = [], [], []
for i in wrong_ids:
    if i in bv_wrong_ids:
        common_wrong_ids.append(i)
    else:
        lora_wrongs.append(i)
for i in bv_wrong_ids:
    if i in wrong_ids:
        continue
    else:
        bv_wrongs.append(i)

common_correct_ids, bv_corrects, lora_corrects = [], [], []
for i in correct_ids:
    if i in bv_correct_ids:
        common_correct_ids.append(i)
    else:
        lora_corrects.append(i)
for i in bv_correct_ids:
    if i in correct_ids:
        continue
    else:
        bv_corrects.append(i)
"correct:", len(lora_corrects), len(bv_corrects), "wrong:", len(lora_wrongs), len(bv_wrongs),

('correct:', 39, 12, 'wrong:', 15, 18)

In [11]:
gt[0]
gt_id_dict = dict()
for i in gt:
    gt_id_dict[i['id']] = i

In [13]:
# specific questions wrongly answered by our method
for i in bv_wrongs:
    print(gt_id_dict[i]["organ"], gt_id_dict[i]["conversations"])

Abdomen [{'from': 'human', 'value': '<image>\nDoes the liver look normal?'}, {'from': 'gpt', 'value': 'Yes'}]
Abdomen [{'from': 'human', 'value': '<image>\nDoes the picture contain spleen?'}, {'from': 'gpt', 'value': 'Yes'}]
Abdomen [{'from': 'human', 'value': '<image>\nDoes this image look normal?'}, {'from': 'gpt', 'value': 'No'}]
Abdomen [{'from': 'human', 'value': '<image>\nDoes the picture contain spleen?'}, {'from': 'gpt', 'value': 'No'}]
Abdomen [{'from': 'human', 'value': '<image>\nDoes the picture contain spleen?'}, {'from': 'gpt', 'value': 'Yes'}]
Lung [{'from': 'human', 'value': '<image>\nCan pneumothorax be observed on the lower left lung in this picture?'}, {'from': 'gpt', 'value': 'No'}]
Lung [{'from': 'human', 'value': '<image>\nIs this image normal?'}, {'from': 'gpt', 'value': 'No'}]
Lung [{'from': 'human', 'value': '<image>\nCan pneumothorax be observed on the lower right lung in this picture?'}, {'from': 'gpt', 'value': 'No'}]
Lung [{'from': 'human', 'value': '<image>

In [14]:
for i in lora_wrongs:
    print(gt_id_dict[i]["organ"], gt_id_dict[i]["conversations"])

Lung [{'from': 'human', 'value': '<image>\nIs the lung healthy?'}, {'from': 'gpt', 'value': 'No'}]
Abdomen [{'from': 'human', 'value': '<image>\nIs the liver healthy?'}, {'from': 'gpt', 'value': 'Yes'}]
Abdomen [{'from': 'human', 'value': '<image>\nDoes this image look normal?'}, {'from': 'gpt', 'value': 'Yes'}]
Abdomen [{'from': 'human', 'value': '<image>\nDoes the picture contain heart?'}, {'from': 'gpt', 'value': 'No'}]
Abdomen [{'from': 'human', 'value': '<image>\nDoes the picture contain liver?'}, {'from': 'gpt', 'value': 'No'}]
Lung [{'from': 'human', 'value': '<image>\nCan effusion be observed on the lower left lung in this picture?'}, {'from': 'gpt', 'value': 'Yes'}]
Neck [{'from': 'human', 'value': '<image>\nDoes the trachea exist in this picture?'}, {'from': 'gpt', 'value': 'No'}]
Brain_Tissue [{'from': 'human', 'value': '<image>\nIs this an MRI image with T1 weighted?'}, {'from': 'gpt', 'value': 'No'}]
Brain_Tissue [{'from': 'human', 'value': '<image>\nIs there evidence of a

In [12]:
bv_corrects
# gt_id_dict[11979]
for i in bv_corrects:
    print(gt_id_dict[i])

{'image': 'xmlab105/source.jpg', 'id': 11961, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nDoes the picture contain heart?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab299/source.jpg', 'id': 12273, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nCan pneumothorax be observed on the lower left lung in this picture?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab314/source.jpg', 'id': 12333, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nCan pneumonia be observed on the right lung in this picture?'}, {'from': 'gpt', 'value': 'Yes'}]}
{'image': 'xmlab361/source.jpg', 'id': 12438, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nCan nodule be observed on the lower right lung in this picture?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab378/source.jpg', 'id': 12496, 'organ': 'L

In [27]:
wrong_after_tuning_ids = list(set(lora_corrects) and set(bv_wrongs))
print(len(wrong_after_tuning_ids), len(lora_corrects), len(bv_wrongs))

correct_after_tuning_ids = list(set(lora_wrongs) and set(bv_corrects))
len(correct_after_tuning_ids), len(lora_wrongs), len(bv_corrects)

18 39 18


(12, 15, 12)

In [19]:
set(wrong_after_tuning_ids) == set(bv_wrongs)

True

In [24]:
# wrong_after_tuning_ids
for i in wrong_after_tuning_ids:
    print(gt_id_dict[i])

{'image': 'xmlab468/source.jpg', 'id': 12642, 'organ': 'Brain_Tissue', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nIs this an MRI image with T1 weighted?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab429/source.jpg', 'id': 12611, 'organ': 'Neck', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nDoes the picture contain the organ which has the effect of discharging waste?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab427/source.jpg', 'id': 12580, 'organ': 'Neck', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nDo the organs in the image exist in the head?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab513/source.jpg', 'id': 12738, 'organ': 'Brain_Tissue', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nIs this an MRI image with T2 weighted?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab622/source.jpg', 'id': 12925, 'organ': 'Pelvic Cav

In [28]:
# correct_after_tuning_ids
for i in correct_after_tuning_ids:
    print(gt_id_dict[i])

{'image': 'xmlab557/source.jpg', 'id': 12833, 'organ': 'Abdomen', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nDoes the picture contain the organ that could be used for secreting enzymes?'}, {'from': 'gpt', 'value': 'Yes'}]}
{'image': 'xmlab102/source.jpg', 'id': 11942, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nIs the lung healthy?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab427/source.jpg', 'id': 12583, 'organ': 'Neck', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nDoes the trachea exist in this picture?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab604/source.jpg', 'id': 12908, 'organ': 'Pelvic Cavity', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nDoes the picture contain the organ which has the effect of discharging waste?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab442/source.jpg', 'id': 12623, 'organ': 